<a href="https://colab.research.google.com/github/bimadewantoro/NER-IndonesianLanguages/blob/main/NER_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import itertools
import pandas as pd
import numpy as np
from datasets import load_dataset
from datasets import load_metric
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import torch

In [3]:
task = "ner"
datasets = load_dataset('id_nergrit_corpus', 'ner')
model = "indolem/indobert-base-uncased"
batch_size = 24

  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
label_list = datasets["train"].features["ner_tags"].feature.names

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForTokenClassification.from_pretrained(model, num_labels=len(label_list))
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

In [6]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [7]:
tokenized_datasets = datasets.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    remove_columns=datasets["train"].column_names,
    load_from_cache_file=False,
)

#0:   0%|          | 0/4 [00:00<?, ?ba/s]

#2:   0%|          | 0/4 [00:00<?, ?ba/s]

#1:   0%|          | 0/4 [00:00<?, ?ba/s]

#3:   0%|          | 0/4 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


...

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [8]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [9]:
metric = load_metric("seqeval")

<ipython-input-9-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [10]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    return metric.compute(predictions=true_predictions, references=true_labels)

In [11]:
args = TrainingArguments(
    "test-ner",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,  # Use mixed-precision training for faster computation
)

In [12]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Using cuda_amp half precision backend


In [13]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 12532
  Num Epochs = 3
  Instantaneous batch size per device = 24
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 1
  Total optimization steps = 1569
  Number of trainable parameters = 109997607
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Crd,Dat,Evt,Fac,Gpe,Lan,Law,Loc,Mon,Nor,Ord,Org,Per,Prc,Prd,Qty,Reg,Tim,Woa,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,0.541400,0.227691,"{'precision': 0.8888059701492538, 'recall': 0.9311962470680218, 'f1': 0.9095074455899198, 'number': 1279}","{'precision': 0.9342783505154639, 'recall': 0.9223918575063613, 'f1': 0.9282970550576184, 'number': 786}","{'precision': 0.5, 'recall': 0.6063348416289592, 'f1': 0.5480572597137013, 'number': 221}","{'precision': 0.2463768115942029, 'recall': 0.34, 'f1': 0.28571428571428575, 'number': 50}","{'precision': 0.8447701532311792, 'recall': 0.8793342579750347, 'f1': 0.8617057424396873, 'number': 1442}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1}","{'precision': 0.38095238095238093, 'recall': 0.449438202247191, 'f1': 0.41237113402061853, 'number': 89}","{'precision': 0.5141509433962265, 'recall': 0.49099099099099097, 'f1': 0.5023041474654377, 'number': 444}","{'precision': 0.9344262295081968, 'recall': 0.9293478260869565, 'f1': 0.9318801089918256, 'number': 184}","{'precision': 0.7037735849056603, 'recall': 0.7415506958250497, 'f1': 0.7221684414327202, 'number': 1006}","{'precision': 0.7555555555555555, 'recall': 0.6538461538461539, 'f1': 0.7010309278350516, 'number': 104}","{'precision': 0.5421209117938554, 'recall': 0.6880503144654088, 'f1': 0.606430155210643, 'number': 795}","{'precision': 0.9230769230769231, 'recall': 0.9386503067484663, 'f1': 0.9307984790874524, 'number': 1956}","{'precision': 0.9695431472081218, 'recall': 0.9744897959183674, 'f1': 0.9720101781170484, 'number': 196}","{'precision': 0.5744897959183674, 'recall': 0.5663983903420523, 'f1': 0.5704154002026343, 'number': 994}","{'precision': 0.711038961038961, 'recall': 0.7064516129032258, 'f1': 0.7087378640776699, 'number': 310}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 53}","{'precision': 0.921875, 'recall': 0.8872180451127819, 'f1': 0.9042145593869733, 'number': 133}","{'precision': 0.25, 'recall': 0.1111111111111111, 'f1': 0.15384615384615383, 'number': 9}",0.772026,0.801134,0.786311,0.933217
2,0.207800,0.203649,"{'precision': 0.9044730856709629, 'recall': 0.9327599687255669, 'f1': 0.9183987682832948, 'number': 1279}","{'precision': 0.9348659003831418, 'recall': 0.9312977099236641, 'f1': 0.9330783938814532, 'number': 786}","{'precision': 0.5597014925373134, 'recall': 0.6787330316742082, 'f1': 0.6134969325153374, 'number': 221}","{'precision': 0.367816091954023, 'recall': 0.64, 'f1': 0.4671532846715328, 'number': 50}","{'precision': 0.8858350951374208, 'recall': 0.8717059639389736, 'f1': 0.8787137364557848, 'number': 1442}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1}","{'precision': 0.39473684210526316, 'recall': 0.5056179775280899, 'f1': 0.4433497536945813, 'number': 89}","{'precision': 0.5555555555555556, 'recall': 0.5855855855855856, 'f1': 0.5701754385964912, 'number': 444}","{'precision': 0.9508196721311475, 'recall': 0.9456521739130435, 'f1': 0.9482288828337875, 'number': 184}","{'precision': 0.7452054794520548, 'recall': 0.8111332007952287, 'f1': 0.7767729652546406, 'number': 1006}","{'precision': 0.7168141592920354, 'recall': 0.7788461538461539, 'f1': 0.7465437788018434, 'number': 104}","{'precision': 0.6543665436654367, 'recall': 0.6691823899371069, 'f1': 0.6616915422885573, 'number': 795}","{'precision': 0.949510561566203, 'recall': 0.9422290388548057, 'f1': 0.9458557865024376, 'number': 1956}","{'precision': 0.9896373056994818, 'recall': 0.9744897959183674, 'f1': 0.9820051413881749, 'number': 196}","{'precision': 0.606494746895893, 'recall': 0.6388329979879276, 'f1': 0.6222439980401764, 'number': 994}","{'precision': 0.7458745874587459, 'recall': 0.7290322580645161, 'f1': 0.7373572593800979, 'number': 310}","{'precision': 0.39705882352941174, 'recall': 0.5094339622641509, 'f1': 0.44628099173553715, 'number': 53}","{'precision': 0.9841269841269841, 'recall': 0.9323308270676691, 'f1': 0.9575289575289575, 'number': 13

Saving model checkpoint to test-ner/checkpoint-500
Configuration saved in test-ner/checkpoint-500/config.json
Model weights saved in test-ner/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-ner/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-ner/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 2521
  Batch size = 24
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "{'precision': 0.8888059701492538, 'recall': 0.9311962470680218, 'f1': 0.9095074455899198, 'number': 1279}" of type <class 'dict'> for key "eval/CRD" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Train

TrainOutput(global_step=1569, training_loss=0.29489816260839286, metrics={'train_runtime': 293.7625, 'train_samples_per_second': 127.981, 'train_steps_per_second': 5.341, 'total_flos': 1544694644641392.0, 'train_loss': 0.29489816260839286, 'epoch': 3.0})

In [33]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 2521
  Batch size = 24


Trainer is attempting to log a value of "{'precision': 0.9079048349961627, 'recall': 0.924941360437842, 'f1': 0.9163439194422929, 'number': 1279}" of type <class 'dict'> for key "eval/CRD" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9439490445859873, 'recall': 0.9427480916030534, 'f1': 0.9433481858688733, 'number': 786}" of type <class 'dict'> for key "eval/DAT" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.5650557620817844, 'recall': 0.6877828054298643, 'f1': 0.6204081632653061, 'number': 221}" of type <class 'dict'> for key "eval/EVT" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.3764705882352941, 'recall': 0.64, 'f1': 0.4740740

{'eval_loss': 0.20862019062042236,
 'eval_CRD': {'precision': 0.9079048349961627,
  'recall': 0.924941360437842,
  'f1': 0.9163439194422929,
  'number': 1279},
 'eval_DAT': {'precision': 0.9439490445859873,
  'recall': 0.9427480916030534,
  'f1': 0.9433481858688733,
  'number': 786},
 'eval_EVT': {'precision': 0.5650557620817844,
  'recall': 0.6877828054298643,
  'f1': 0.6204081632653061,
  'number': 221},
 'eval_FAC': {'precision': 0.3764705882352941,
  'recall': 0.64,
  'f1': 0.47407407407407404,
  'number': 50},
 'eval_GPE': {'precision': 0.8895027624309392,
  'recall': 0.8932038834951457,
  'f1': 0.8913494809688581,
  'number': 1442},
 'eval_LAN': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1},
 'eval_LAW': {'precision': 0.4032258064516129,
  'recall': 0.5617977528089888,
  'f1': 0.4694835680751174,
  'number': 89},
 'eval_LOC': {'precision': 0.5759637188208617,
  'recall': 0.5720720720720721,
  'f1': 0.5740112994350283,
  'number': 444},
 'eval_MON': {'precision': 0.950

In [15]:
trainer.save_model("NER_FineTunebyBimaDewantoro")

Saving model checkpoint to NER_FineTunebyBimaDewantoro
Configuration saved in NER_FineTunebyBimaDewantoro/config.json
Model weights saved in NER_FineTunebyBimaDewantoro/pytorch_model.bin
tokenizer config file saved in NER_FineTunebyBimaDewantoro/tokenizer_config.json
Special tokens file saved in NER_FineTunebyBimaDewantoro/special_tokens_map.json


In [21]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [29]:
!cp -R /content/NER_FineTunebyBimaDewantoro /content/gdrive/MyDrive/Oche

AttributeError: ignored